In [ ]:
!pip3 install ftfy
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/Shareddrives/AI Team Docs/Version Releases/Version 0.9/Repo/v-diffusion-pytorch

# Add the Text String Here:

In [ ]:
#@markdown #**Parameters(Diffusion)**
texts = "fantasy world" #@param {type:"string"}
Image_Size =  256#@param {type:"number"}
clip_guidance_scale =  500#@param {type:"number"}
Steps =  300#@param {type:"number"}
seed = 200#@param {type:"number"}
# generated_image_name = "/content/gdrive/MyDrive/NFT_Test/Gen_Images/Clip_blue abortion.png"#@param {type:"string"}
generated_image_name = "/content//Clip_"+texts+str(clip_guidance_scale)+'_'+str(Steps)+'.png'

In [ ]:
#@markdown #**CLIP diffusion**
from functools import partial
from pathlib import Path

from PIL import Image
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm import trange

from CLIP import clip
from diffusion import get_model, get_models, sampling, utils
torch.cuda.empty_cache()
checkpoint = 'checkpoints/cc12m_1_cfg.pth'
MODEL = 'cc12m_1'
Steps = Steps
cutn = 16
cut_pow = 1.
prompt = texts
clip_guidance_scale = clip_guidance_scale
batch_size = 1
Size = (Image_Size, Image_Size)
seed = seed
n = 1

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([]) ** self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutout = F.adaptive_avg_pool2d(cutout, self.cut_size)
            cutouts.append(cutout)
        return torch.cat(cutouts)


def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)


def parse_prompt(prompt):
    if prompt.startswith('http://') or prompt.startswith('https://'):
        vals = prompt.rsplit(':', 2)
        vals = [vals[0] + ':' + vals[1], *vals[2:]]
    else:
        vals = prompt.rsplit(':', 1)
    vals = vals + ['', '1'][len(vals):]
    return vals[0], float(vals[1])


def resize_and_center_crop(image, size):
    fac = max(size[0] / image.size[0], size[1] / image.size[1])
    image = image.resize((int(fac * image.size[0]), int(fac * image.size[1])), Image.LANCZOS)
    return TF.center_crop(image, size[::-1])


def make_cond_model_fn(model, cond_fn):
    def cond_model_fn(x, t, **extra_args):
        with torch.enable_grad():
            x = x.detach().requires_grad_()
            v = model(x, t, **extra_args)
            alphas, sigmas = utils.t_to_alpha_sigma(t)
            pred = x * alphas[:, None, None, None] - v * sigmas[:, None, None, None]
            cond_grad = cond_fn(x, t, pred, **extra_args).detach()
            v = v.detach() - cond_grad * (sigmas[:, None, None, None] / alphas[:, None, None, None])
        return v

    return cond_model_fn


def generate_image():
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)

    model = get_model(MODEL)()
    _, side_y, side_x = model.shape
    if Size:
        side_x, side_y = Size
    model.load_state_dict(torch.load(checkpoint, map_location='cpu'))
    model = model.to(device).eval().requires_grad_(False)
    clip_model_name = model.clip_model if hasattr(model, 'clip_model') else 'ViT-B/16'
    clip_model = clip.load(clip_model_name, jit=False, device=device)[0]
    clip_model.eval().requires_grad_(False)
    normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                     std=[0.26862954, 0.26130258, 0.27577711])
    make_cutouts = MakeCutouts(clip_model.visual.input_resolution, cutn, cut_pow)


    txt, weights = parse_prompt(prompt.replace(']', ''))
    target_embeds = clip_model.encode_text(clip.tokenize(txt).to(device)).float()

    weights = torch.tensor(weights, device=device)
    if weights.sum().abs() < 1e-3:
        raise RuntimeError('The weights must not sum to 0.')
    weights /= weights.sum().abs()

    clip_embed = F.normalize(target_embeds.mul(weights).sum(0, keepdim=True), dim=-1)
    clip_embed = clip_embed.repeat([n, 1])

    torch.manual_seed(seed)

    def cond_fn(x, t, pred, clip_embed):
        if min(pred.shape[2:4]) < 256:
            pred = F.interpolate(pred, scale_factor=2, mode='bilinear', align_corners=False)
        clip_in = normalize(make_cutouts((pred + 1) / 2))
        image_embeds = clip_model.encode_image(clip_in).view([cutn, x.shape[0], -1])
        losses = spherical_dist_loss(image_embeds, clip_embed[None])
        loss = losses.mean(0).sum() * clip_guidance_scale
        grad = -torch.autograd.grad(loss, x)[0]
        return grad

    def run(x, steps, clip_embed):
        if hasattr(model, 'clip_model'):
            extra_args = {'clip_embed': clip_embed}
            cond_fn_ = cond_fn
        else:
            extra_args = {}
            cond_fn_ = partial(cond_fn, clip_embed=clip_embed)
        if clip_guidance_scale:
            model_fn = make_cond_model_fn(model, cond_fn_)
        else:
            model_fn = model

        return sampling.sample(model_fn, x, steps, 1., extra_args)

    def run_all(n, batch_size):
        x = torch.randn([n, 3, side_y, side_x], device=device)
        t = torch.linspace(1, 0, Steps + 1, device=device)[:-1]
        if model.min_t == 0:
            steps = utils.get_spliced_ddpm_cosine_schedule(t)
        else:
            steps = utils.get_ddpm_schedule(t)
        for i in trange(0, n, batch_size):
            cur_batch_size = min(n - i, batch_size)

            outs = run(x[i:i + cur_batch_size], steps, clip_embed[i:i + cur_batch_size])
            for j, out in enumerate(outs):
                utils.to_pil_image(out).save(generated_image_name)

    try:
        run_all(n, batch_size)
    except KeyboardInterrupt:
        pass


In [ ]:
generate_image()
torch.cuda.empty_cache()

# NO NEED TO RUN THE CODE BELOW

In [ ]:
#@markdown #**Parameters(Neural Style Transfer)**
style_path = "/content/" #@param {type:"string"}
content_path = "/content/" #@param {type:"string"}
result_path = "/content/" #@param {type:"string"}
Output_Image_Size =  1024#@param {type:"number"}
iterations=1000 #@param{type:"number"}

In [ ]:
#@markdown #**Neural Style Transfer**
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (10,10)
mpl.rcParams['axes.grid'] = False

import numpy as np
from PIL import Image
import time
import functools
import os
%tensorflow_version 1.x
import tensorflow as tf

# from tensorflow.python.keras.preprocessing import image as kp_image
# from tensorflow.python.keras import models 
# from tensorflow.python.keras import losses
# from tensorflow.python.keras import layers
# from tensorflow.python.keras import backend as K

# # max_dim = Output_Image_Size
# tf.enable_eager_execution()
# print("Eager execution: {}".format(tf.executing_eagerly()))

# # Content layer where will pull our feature maps
# content_layers = ['block5_conv2'] 

# # Style layer we are interested in
# style_layers = ['block1_conv1',
#                 'block2_conv1',
#                 'block3_conv1', 
#                 'block4_conv1', 
#                 'block5_conv1'
#                ]

# num_content_layers = len(content_layers)
# num_style_layers = len(style_layers)
# max_dim = Output_Image_Size

def load_img(path_to_img):
  
  img = Image.open(path_to_img)
  long = max(img.size)
  scale = max_dim/long
  img = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)
  
  img = kp_image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  return img

def imshow(img, title=None):
  # Remove the batch dimension
  out = np.squeeze(img, axis=0)
  # Normalize for display 
  out = out.astype('uint8')
  plt.imshow(out)
  if title is not None:
    plt.title(title)
  plt.imshow(out)


def load_and_process_img(path_to_img):
  img = load_img(path_to_img)
  img = tf.keras.applications.vgg19.preprocess_input(img)
  return img

# def deprocess_img(processed_img):
#   x = processed_img.copy()
#   if len(x.shape) == 4:
#     x = np.squeeze(x, 0)
#   assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
#                              "dimension [1, height, width, channel] or [height, width, channel]")
#   if len(x.shape) != 3:
#     raise ValueError("Invalid input to deprocessing image")
  
#   # perform the inverse of the preprocessing step
#   x[:, :, 0] += 103.939
#   x[:, :, 1] += 116.779
#   x[:, :, 2] += 123.68
#   x = x[:, :, ::-1]

#   x = np.clip(x, 0, 255).astype('uint8')
#   return x

# def get_model():
#   # Load our model. We load pretrained VGG, trained on data
#   vgg = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet')
#   vgg.trainable = False
#   # Get output layers corresponding to style and content layers 
#   style_outputs = [vgg.get_layer(name).output for name in style_layers]
#   content_outputs = [vgg.get_layer(name).output for name in content_layers]
#   model_outputs = style_outputs + content_outputs
#   # Build model 
#   return models.Model(vgg.input, model_outputs)

# def get_content_loss(base_content, target):
#   return tf.reduce_mean(tf.square(base_content - target))

# def gram_matrix(input_tensor):
#   # We make the image channels first 
#   channels = int(input_tensor.shape[-1])
#   a = tf.reshape(input_tensor, [-1, channels])
#   n = tf.shape(a)[0]
#   gram = tf.matmul(a, a, transpose_a=True)
#   return gram / tf.cast(n, tf.float32)

# def get_style_loss(base_style, gram_target):
#   """Expects two images of dimension h, w, c"""
#   # height, width, num filters of each layer
#   # We scale the loss at a given layer by the size of the feature map and the number of filters
#   height, width, channels = base_style.get_shape().as_list()
#   gram_style = gram_matrix(base_style)
  
#   return tf.reduce_mean(tf.square(gram_style - gram_target))# / (4. * (channels ** 2) * (width * height) ** 2)


# def get_feature_representations(model, content_path, style_path):
#   # Load our images in 
#   content_image = load_and_process_img(content_path)
#   style_image = load_and_process_img(style_path)
  
#   # batch compute content and style features
#   style_outputs = model(style_image)
#   content_outputs = model(content_image)
  
  
#   # Get the style and content feature representations from our model  
#   style_features = [style_layer[0] for style_layer in style_outputs[:num_style_layers]]
#   content_features = [content_layer[0] for content_layer in content_outputs[num_style_layers:]]
#   return style_features, content_features

# def compute_loss(model, loss_weights, init_image, gram_style_features, content_features):
#   """This function will compute the loss total loss.
  
#   Arguments:
#     model: The model that will give us access to the intermediate layers
#     loss_weights: The weights of each contribution of each loss function. 
#       (style weight, content weight, and total variation weight)
#     init_image: Our initial base image. This image is what we are updating with 
#       our optimization process. We apply the gradients wrt the loss we are 
#       calculating to this image.
#     gram_style_features: Precomputed gram matrices corresponding to the 
#       defined style layers of interest.
#     content_features: Precomputed outputs from defined content layers of 
#       interest.
      
#   Returns:
#     returns the total loss, style loss, content loss, and total variational loss
#   """
#   style_weight, content_weight = loss_weights
  
#   # Feed our init image through our model. This will give us the content and 
#   # style representations at our desired layers. Since we're using eager
#   # our model is callable just like any other function!
#   model_outputs = model(init_image)
  
#   style_output_features = model_outputs[:num_style_layers]
#   content_output_features = model_outputs[num_style_layers:]
  
#   style_score = 0
#   content_score = 0

#   # Accumulate style losses from all layers
#   # Here, we equally weight each contribution of each loss layer
#   weight_per_style_layer = 1.0 / float(num_style_layers)
#   for target_style, comb_style in zip(gram_style_features, style_output_features):
#     style_score += weight_per_style_layer * get_style_loss(comb_style[0], target_style)
    
#   # Accumulate content losses from all layers 
#   weight_per_content_layer = 1.0 / float(num_content_layers)
#   for target_content, comb_content in zip(content_features, content_output_features):
#     content_score += weight_per_content_layer* get_content_loss(comb_content[0], target_content)
  
#   style_score *= style_weight
#   content_score *= content_weight

#   # Get total loss
#   loss = style_score + content_score 
#   return loss, style_score, content_score

# def compute_grads(cfg):
#   with tf.GradientTape() as tape: 
#     all_loss = compute_loss(**cfg)
#   # Compute gradients wrt input image
#   total_loss = all_loss[0]
#   return tape.gradient(total_loss, cfg['init_image']), all_loss


# import IPython.display

# def run_style_transfer(content_path, 
#                        style_path,
#                        num_iterations=1000,
#                        content_weight=1e3, 
#                        style_weight=1e-2): 
#   # We don't need to (or want to) train any layers of our model, so we set their
#   # trainable to false. 
#   model = get_model() 
#   for layer in model.layers:
#     layer.trainable = False
  
#   # Get the style and content feature representations (from our specified intermediate layers) 
#   style_features, content_features = get_feature_representations(model, content_path, style_path)
#   gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]
  
#   # Set initial image
#   init_image = load_and_process_img(content_path)
#   init_image = tf.Variable(init_image, dtype=tf.float32)
#   # Create our optimizer
#   opt = tf.train.AdamOptimizer(learning_rate=5, beta1=0.99, epsilon=1e-1)

#   # For displaying intermediate images 
#   iter_count = 1
  
#   # Store our best result
#   best_loss, best_img = float('inf'), None
  
#   # Create a nice config 
#   loss_weights = (style_weight, content_weight)
#   cfg = {
#       'model': model,
#       'loss_weights': loss_weights,
#       'init_image': init_image,
#       'gram_style_features': gram_style_features,
#       'content_features': content_features
#   }
    
#   # For displaying
#   num_rows = 2
#   num_cols = 5
#   display_interval = num_iterations/(num_rows*num_cols)
#   start_time = time.time()
#   global_start = time.time()
  
#   norm_means = np.array([103.939, 116.779, 123.68])
#   min_vals = -norm_means
#   max_vals = 255 - norm_means   
  
#   imgs = []
#   for i in range(num_iterations):
#     grads, all_loss = compute_grads(cfg)
#     loss, style_score, content_score = all_loss
#     opt.apply_gradients([(grads, init_image)])
#     clipped = tf.clip_by_value(init_image, min_vals, max_vals)
#     init_image.assign(clipped)
#     end_time = time.time() 
    
#     if loss < best_loss:
#       # Update best loss and best image from total loss. 
#       best_loss = loss
#       best_img = deprocess_img(init_image.numpy())

#     if i % display_interval== 0:
#       start_time = time.time()
      
#       # Use the .numpy() method to get the concrete numpy array
#       plot_img = init_image.numpy()
#       plot_img = deprocess_img(plot_img)
#       imgs.append(plot_img)
#       IPython.display.clear_output(wait=True)
#       IPython.display.display_png(Image.fromarray(plot_img))
#       print('Iteration: {}'.format(i))        
#       print('Total loss: {:.4e}, ' 
#             'style loss: {:.4e}, '
#             'content loss: {:.4e}, '
#             'time: {:.4f}s'.format(loss, style_score, content_score, time.time() - start_time))
#   print('Total time: {:.4f}s'.format(time.time() - global_start))
#   IPython.display.clear_output(wait=True)
#   plt.figure(figsize=(14,4))
#   for i,img in enumerate(imgs):
#       plt.subplot(num_rows,num_cols,i+1)
#       plt.imshow(img)
#       plt.xticks([])
#       plt.yticks([])
      
#   return best_img, best_loss 

def show_results(best_img, content_path, style_path, show_large_final=True):
  plt.figure(figsize=(10, 5))
  content = load_img(content_path) 
  style = load_img(style_path)

  plt.subplot(1, 2, 1)
  imshow(content, 'Content Image')

  plt.subplot(1, 2, 2)
  imshow(style, 'Style Image')

  if show_large_final: 
    plt.figure(figsize=(10, 10))

    plt.imshow(best_img)
    plt.title('Output Image')
    plt.show()

In [ ]:
#@markdown #**Generate Result**
# Set up some global values here

save_file_path = result_path
plt.figure(figsize=(10,10))

content = load_img(content_path).astype('uint8')
style = load_img(style_path).astype('uint8')

plt.subplot(1, 2, 1)
imshow(content, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style, 'Style Image')
plt.show()
best, best_loss = run_style_transfer(content_path, style_path, 
                                     num_iterations=iterations,content_weight=1e-7,style_weight=1)
result = Image.fromarray(best)
result.save(save_file_path)
show_results(best, content_path, style_path)

In [ ]:
!nvidia-smi